# Dummy Column

Using dask futures API to pass cuDF column and Series objects between workers. Columns work but Series do not.

In [1]:
from numba import cuda
from distributed.protocol.cuda import cuda_serialize, cuda_deserialize

class DummyColumn(object):                                                                                                                                                                                      
                                                                                                                                                                                                                
    def __init__(self):                                                                                                                                                                                         
        import threading                                                                                                                                                                                        
        self.lock = threading.Lock()                                                                                                                                                                            
        pass                                                                                                                                                                                                    
                                                                                                                                                                                                                
    def serialize(self):                                                                                                                                                                                        
        header = {}                                                                                                                                                                                             
        frames = [cuda.to_device(np.arange(100000))]                                                                                                                                                            
        return header, frames                                                                                                                                                                                   
                                                                                                                                                                                                                
    @classmethod                                                                                                                                                                                                
    def deserialize(cls, header, frames):                                                                                                                                                                       
        return np.array([1, 2, 3])                                                                                                                                                                              
                                                                                                                                                                                                                
@cuda_serialize.register(DummyColumn)                                                                                                                                                                           
def serialize(col):                                                                                                                                                                                             
    return col.serialize()                                                                                                                                                                                      
                                                                                                                                                                                                                
@cuda_deserialize.register(DummyColumn)                                                                                                                                                                         
def deserialize(header, frames):                                                                                                                                                                                
    return DummyColumn.deserialize(header, frames) 

In [2]:
import dask
import distributed
import pandas as pd
import os
import numpy as np

In [3]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')

Dask: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/dask/__init__.py
Distributed: /home/nfs/ashwint/workspace/distributed/distributed/__init__.py


In [4]:
base_env = {
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "2,3",
}
os.environ.update(base_env)

In [5]:
from dask.distributed import Client, wait
from dask_cuda import DGX

#cluster = DGX(CUDA_VISIBLE_DEVICES=[2,3], 
#              dashboard_address='10.33.227.165:8789')
#client = Client(cluster)
client = Client('ucx://10.33.225.165:8786')
client

Client Scheduler: ucx://10.33.225.165:8786 Dashboard: http://10.33.225.165:8787/status,Cluster Workers: 2 Cores: 2 Memory: 64.00 GB


In [6]:
worker_1, worker_2 = client.scheduler_info()['workers']

In [ ]:
from dask.distributed import wait
import operator

left = client.map(lambda x: DummyColumn(), range(1000), workers=[worker_1])
right = client.map(lambda x: DummyColumn(), range(1000), workers=[worker_2])
results = client.map(lambda x,y: (x,y), left, right)
_ = wait(results)